In [280]:
# in the name of God
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [281]:
train = pd.read_csv('ML_HW2_Question7/train.csv')
test = pd.read_csv('ML_HW2_Question7/test.csv')
y = train.pop('SalePrice')
test_id = test.pop('Id').values
train_id = train.pop('Id')

In [282]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(train['Condition1'])
list(le.classes_)


['Artery', 'Feedr', 'Norm', 'PosA', 'PosN', 'RRAe', 'RRAn', 'RRNe', 'RRNn']

In [283]:
def preprocess_matrix(matrix):
    cols = matrix.columns
    types = matrix.dtypes
    obj_columns = [cols[i] for i in range(len(cols)) if types[i] == 'object']
    preprocessed_matrix = pd.get_dummies(matrix, columns = obj_columns, prefix=obj_columns)

    return preprocessed_matrix.values, preprocessed_matrix.columns

In [299]:
preprocessed_train, train_cols = preprocess_matrix(train)
n, m = preprocessed_train.shape
matrix = np.ones((n,m+1))
matrix[:, 1:] = preprocessed_train
preprocessed_test, test_cols = preprocess_matrix(test)



where_are_NaNs = np.isnan(matrix)
matrix[where_are_NaNs] = 0

In [285]:

def calculate_error_sse(w, y, matrix):
    xw = np.dot(matrix, w)
    return np.sum(np.subtract(xw, y)**2)

def calculate_w(matrix, y):
    w3 = np.dot(np.dot(la.pinv(np.dot(matrix.T, matrix)),matrix.T), y)
    return w3


def calculate_w_l2(matrix, y, landa):
    w = np.dot(np.dot(la.pinv(np.add(np.dot(matrix.T, matrix), landa * np.identity(matrix.shape[1]))),matrix.T), y)
    return w

In [286]:
# shuffled_train = train.sample(frac=1).reset_index(drop=True)
import numpy.linalg as la
offset = matrix.shape[0]//10
ranges = [10**i for i in range(-10,11)]
sses = []
ws = []
for j in ranges:
    sse_sum = 0
    ws_sum = 0
    for i in range(0, matrix.shape[0], offset):
        train_matrix = np.concatenate((matrix[0:i], matrix[i+offset:]))
        train_y = np.concatenate((y[0:i],y[i+offset:]))
        
        validation_matrix = matrix[i:i+offset]
        validation_y = y[i:i+offset]
        w = calculate_w_l2(train_matrix, train_y, j)
        ws_sum += w
        sse = calculate_error_sse(w, validation_y, validation_matrix)
        sse_sum += sse
    sse_average = sse_sum / 5
    w_average = ws_sum / 5
    sses.append(sse_average)
    ws.append(w_average)

In [287]:
min_sse = min([i for i in sses if i != min(sses)])
print(sses)
min_sse_index = sses.index(min_sse)

landa =  ranges[min_sse_index]
print(landa)
w = calculate_w_l2(matrix, y, landa) 

[704778994887.6139, 704771150721.634, 704595837681.3785, 703773829725.8079, 704442157096.0166, 694394106593.0865, 615072401937.0681, 421583280695.15857, 378538019061.33606, 359973235298.8895, 320947238509.3831, 298489118199.14154, 311007034686.7234, 378086899869.1984, 469752389260.8617, 503561580415.54767, 564918599660.8558, 605961522417.1735, 659195511026.9766, 979772940890.8438, 2193679016023.9426]
100


In [288]:
preprocessed_test, felan = preprocess_matrix(test)
print(preprocessed_test.shape)
print(w.shape)
print(preprocessed_train.shape)

(1459, 270)
(289,)
(1460, 288)


In [289]:
new_w = np.zeros((preprocessed_test.shape[1]+1,))
j = 1
new_w[0] = w[0]
for i in range(len(train_cols)):
    if train_cols[i] in test_cols:
        new_w[j] = w[i+1]
        j+=1


In [290]:
n, m = preprocessed_test.shape
matrix = np.ones((n,m+1))
matrix[:, 1:] = preprocessed_test

test_y = np.dot(matrix, new_w)

where_are_NaNs = np.isnan(test_y)
test_y[where_are_NaNs] = np.average([test_y[i] for i in range(len(test_y)) if not where_are_NaNs[i]])

In [291]:
import pandas as pd
percentile_list = pd.DataFrame(
    {'Id': test_id,
     'SalePrice': test_y,
    })
percentile_list.to_csv('resule.csv', sep=',', index=False)